In [3]:
import numpy as np
import pandas as pd
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Класс для извлечения признаков из ARP-пакетов
class ARPFeatureExtractor:
    def __init__(self, window_size=10, history_window=60):
        self.window_size = window_size  # окно в секундах для текущих метрик
        self.history_window = history_window  # окно для исторических данных
        self.mac_ip_mapping = {}  # словарь для отслеживания MAC-IP связей
        self.ip_mac_mapping = {}  # словарь для отслеживания IP-MAC связей
        self.packet_history = []  # история пакетов
        self.gateway_mac = None  # MAC-адрес шлюза (определяется автоматически)
        
    def extract_features(self, packets):
        """Извлечение признаков из набора пакетов"""
        self.packet_history.extend(packets)
        # Оставляем только пакеты в пределах истории
        current_time = max([p['timestamp'] for p in packets])
        self.packet_history = [p for p in self.packet_history 
                              if current_time - p['timestamp'] <= self.history_window]
        
        # Определяем шлюз, если еще не определен
        if not self.gateway_mac:
            self._identify_gateway()
        
        # Вычисляем базовые признаки
        features = {}
        window_packets = [p for p in self.packet_history 
                         if current_time - p['timestamp'] <= self.window_size]
        
        # 1. Дубликаты
        features['duplicate_count'] = self._count_duplicates(window_packets)
        features['duplicate_ratio'] = features['duplicate_count'] / len(window_packets) if window_packets else 0
        
        # 2. Два IP имеют один MAC
        self._update_mappings(window_packets)
        features['ips_per_mac'] = self._calculate_ips_per_mac()
        features['ip_mac_entropy'] = self._calculate_ip_mac_entropy()
        
        # 3. Код операции
        req_count = sum(1 for p in window_packets if p['opcode'] == 1)
        rep_count = sum(1 for p in window_packets if p['opcode'] == 2)
        features['request_reply_ratio'] = req_count / rep_count if rep_count else float('inf')
        
        # 4. Отсутствие запроса, а чисто ответ
        features['unsolicited_replies'] = self._count_unsolicited_replies(window_packets)
        features['unsolicited_reply_ratio'] = features['unsolicited_replies'] / rep_count if rep_count else 0
        
        # 5. Высокий объём трафика
        features['packets_per_second'] = len(window_packets) / self.window_size
        features['traffic_zscore'] = self._calculate_traffic_zscore(features['packets_per_second'])
        
        # 6. Частые широковещательные письма не от шлюза
        features['non_gateway_broadcasts'] = self._count_non_gateway_broadcasts(window_packets)
        features['broadcast_entropy'] = self._calculate_broadcast_entropy(window_packets)
        
        # ---- ГРАФОВЫЕ МЕТРИКИ ----
        graph_features = self._calculate_graph_metrics(window_packets)
        features.update(graph_features)
        
        return features
    
    def _identify_gateway(self):
        """Определение MAC-адреса шлюза на основе активности"""
        # В простейшем случае - MAC с наибольшим количеством ARP ответов
        mac_reply_count = {}
        for p in self.packet_history:
            if p['opcode'] == 2:  # ARP ответ
                mac = p['src_mac']
                mac_reply_count[mac] = mac_reply_count.get(mac, 0) + 1
        
        if mac_reply_count:
            self.gateway_mac = max(mac_reply_count, key=mac_reply_count.get)
    
    def _count_duplicates(self, packets):
        """Подсчет дубликатов пакетов"""
        packet_signatures = []
        duplicates = 0
        
        for p in packets:
            # Создаем подпись пакета (ключевые поля)
            sig = (p['src_mac'], p['dst_mac'], p['src_ip'], p['dst_ip'], p['opcode'])
            if sig in packet_signatures:
                duplicates += 1
            else:
                packet_signatures.append(sig)
        
        return duplicates
    
    def _update_mappings(self, packets):
        """Обновление отображений MAC-IP и IP-MAC"""
        for p in packets:
            src_mac, src_ip = p['src_mac'], p['src_ip']
            
            # Обновляем MAC -> IP
            if src_mac not in self.mac_ip_mapping:
                self.mac_ip_mapping[src_mac] = set()
            self.mac_ip_mapping[src_mac].add(src_ip)
            
            # Обновляем IP -> MAC
            if src_ip not in self.ip_mac_mapping:
                self.ip_mac_mapping[src_ip] = set()
            self.ip_mac_mapping[src_ip].add(src_mac)
    
    def _calculate_ips_per_mac(self):
        """Среднее количество IP на один MAC"""
        if not self.mac_ip_mapping:
            return 0
        return sum(len(ips) for ips in self.mac_ip_mapping.values()) / len(self.mac_ip_mapping)
    
    def _calculate_ip_mac_entropy(self):
        """Энтропия распределения IP для MAC"""
        if not self.mac_ip_mapping:
            return 0
        
        # Распределение количества IP на MAC
        ip_counts = [len(ips) for ips in self.mac_ip_mapping.values()]
        total = sum(ip_counts)
        if total == 0:
            return 0
        
        # Вычисляем энтропию
        entropy = 0
        for count in ip_counts:
            prob = count / total
            if prob > 0:
                entropy -= prob * np.log2(prob)
        
        return entropy
    
    def _count_unsolicited_replies(self, packets):
        """Подсчет ответов без соответствующих запросов"""
        # Создаем словарь запросов (target_ip -> source_ip)
        requests = {}
        unsolicited = 0
        
        # Сначала собираем все запросы
        for p in packets:
            if p['opcode'] == 1:  # ARP запрос
                key = p['dst_ip']
                if key not in requests:
                    requests[key] = []
                requests[key].append(p['src_ip'])
        
        # Теперь проверяем ответы
        for p in packets:
            if p['opcode'] == 2:  # ARP ответ
                src_ip = p['src_ip']
                if src_ip not in requests or not requests[src_ip]:
                    unsolicited += 1
        
        return unsolicited
    
    def _calculate_traffic_zscore(self, current_rate):
        """Вычисление Z-score текущей скорости трафика"""
        # Получаем исторические данные о скорости
        if len(self.packet_history) < 2:
            return 0
        
        # Разбиваем историю на окна
        window_packets = {}
        latest_time = max([p['timestamp'] for p in self.packet_history])
        
        # Группируем пакеты по окнам
        for p in self.packet_history:
            window_idx = int((latest_time - p['timestamp']) / self.window_size)
            if window_idx not in window_packets:
                window_packets[window_idx] = 0
            window_packets[window_idx] += 1
        
        # Вычисляем скорости для окон
        rates = [count / self.window_size for count in window_packets.values()]
        
        # Вычисляем среднее и стандартное отклонение
        if not rates:
            return 0
        mean_rate = sum(rates) / len(rates)
        std_rate = np.std(rates) if len(rates) > 1 else 1
        
        # Защита от деления на ноль
        if std_rate == 0:
            std_rate = 1
        
        # Возвращаем Z-score
        return (current_rate - mean_rate) / std_rate
    
    def _count_non_gateway_broadcasts(self, packets):
        """Подсчет широковещательных пакетов не от шлюза"""
        count = 0
        for p in packets:
            if p['is_broadcast'] and p['src_mac'] != self.gateway_mac:
                count += 1
        return count
    
    def _calculate_broadcast_entropy(self, packets):
        """Энтропия интервалов между широковещательными пакетами"""
        broadcasts = [p for p in packets if p['is_broadcast']]
        if len(broadcasts) < 2:
            return 0
        
        # Сортируем по времени
        broadcasts.sort(key=lambda p: p['timestamp'])
        
        # Вычисляем интервалы
        intervals = [broadcasts[i+1]['timestamp'] - broadcasts[i]['timestamp'] 
                     for i in range(len(broadcasts)-1)]
        
        # Делаем дискретные bin-ы для интервалов
        max_interval = max(intervals)
        if max_interval == 0:
            return 0
            
        bins = 10  # количество корзин
        hist, _ = np.histogram(intervals, bins=bins, range=(0, max_interval), density=True)
        
        # Вычисляем энтропию
        entropy = 0
        for prob in hist:
            if prob > 0:
                entropy -= prob * np.log2(prob)
        
        return entropy
    
    # ---- ГРАФОВЫЕ МЕТРИКИ ----
    def _calculate_graph_metrics(self, packets):
        """Вычисление графовых метрик для анализа топологии сети"""
        G = nx.Graph()
        
        # Создаем граф из ARP-пакетов
        for p in packets:
            src = p['src_mac']
            dst = p['dst_mac']
            G.add_node(src, type='mac')
            G.add_node(dst, type='mac')
            G.add_edge(src, dst, timestamp=p['timestamp'], opcode=p['opcode'])
        
        if len(G) <= 1:
            return {
                'graph_density': 0,
                'avg_clustering': 0,
                'avg_degree': 0,
                'degree_centrality_entropy': 0,
                'gateway_betweenness': 0,
                'community_modularity': 0
            }
        
        metrics = {}
        
        # 1. Плотность графа - показывает насколько связаны устройства
        # Высокая плотность при атаке может указывать на сканирование сети
        metrics['graph_density'] = nx.density(G)
        
        # 2. Средний кластерный коэффициент - насколько связаны соседи узлов
        # При атаке часто снижается, т.к. атакующий нарушает нормальную структуру
        metrics['avg_clustering'] = nx.average_clustering(G)
        
        # 3. Средняя степень узла - сколько в среднем связей у устройства
        # Атакующие устройства часто имеют аномально высокую степень
        degrees = [d for _, d in G.degree()]
        metrics['avg_degree'] = sum(degrees) / len(degrees) if degrees else 0
        
        # 4. Энтропия центральности по степени - равномерность распределения связей
        # Атаки часто вызывают аномалии в распределении связей
        degree_cent = nx.degree_centrality(G)
        values = list(degree_cent.values())
        if values:
            hist, _ = np.histogram(values, bins=10, density=True)
            metrics['degree_centrality_entropy'] = -sum(p * np.log2(p) if p > 0 else 0 for p in hist)
        else:
            metrics['degree_centrality_entropy'] = 0
        
        # 5. Промежуточная центральность шлюза - насколько шлюз является мостом между другими узлами
        # При атаке часто снижается, т.к. атакующий перенаправляет трафик через себя
        if self.gateway_mac and self.gateway_mac in G:
            betweenness = nx.betweenness_centrality(G)
            metrics['gateway_betweenness'] = betweenness.get(self.gateway_mac, 0)
        else:
            metrics['gateway_betweenness'] = 0
        
        # 6. Модулярность сообществ - насколько чётко разделена сеть на группы устройств
        # Атаки могут нарушать естественную структуру сообществ
        try:
            communities = nx.community.greedy_modularity_communities(G)
            if communities:
                metrics['community_modularity'] = nx.community.modularity(G, communities)
            else:
                metrics['community_modularity'] = 0
        except:
            metrics['community_modularity'] = 0
        
        return metrics

# Модель автоэнкодера с классификатором
class ARPAttackDetector(nn.Module):
    def __init__(self, input_dim, latent_dim=16, hidden_dims=[64, 32]):
        super(ARPAttackDetector, self).__init__()
        
        # Энкодер
        encoder_layers = []
        prev_dim = input_dim
        for dim in hidden_dims:
            encoder_layers.append(nn.Linear(prev_dim, dim))
            encoder_layers.append(nn.ReLU())
            encoder_layers.append(nn.BatchNorm1d(dim))
            prev_dim = dim
        
        encoder_layers.append(nn.Linear(prev_dim, latent_dim))
        self.encoder = nn.Sequential(*encoder_layers)
        
        # Декодер
        decoder_layers = []
        prev_dim = latent_dim
        for dim in reversed(hidden_dims):
            decoder_layers.append(nn.Linear(prev_dim, dim))
            decoder_layers.append(nn.ReLU())
            decoder_layers.append(nn.BatchNorm1d(dim))
            prev_dim = dim
        
        decoder_layers.append(nn.Linear(prev_dim, input_dim))
        self.decoder = nn.Sequential(*decoder_layers)
        
        # Классификатор
        self.classifier = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.3),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.BatchNorm1d(16),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        # Кодирование входа
        z = self.encoder(x)
        
        # Декодирование
        x_recon = self.decoder(z)
        
        # Классификация
        pred = self.classifier(z)
        
        return x_recon, pred
    
    def get_latent(self, x):
        """Получение латентного представления"""
        return self.encoder(x)

# Функция обучения модели
def train_model(X_train, y_train, input_dim, epochs=100, batch_size=64):
    # Преобразуем данные в тензоры
    X_tensor = torch.FloatTensor(X_train)
    y_tensor = torch.FloatTensor(y_train).view(-1, 1)
    
    # Создаем датасет и загрузчик
    dataset = TensorDataset(X_tensor, y_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Инициализируем модель
    model = ARPAttackDetector(input_dim)
    
    # Функции потерь и оптимизатор
    reconstruction_criterion = nn.MSELoss()
    classification_criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Обучение
    for epoch in range(epochs):
        total_loss = 0
        recon_loss_sum = 0
        class_loss_sum = 0
        
        for X_batch, y_batch in dataloader:
            # Прямой проход
            X_recon, y_pred = model(X_batch)
            
            # Вычисление потерь
            recon_loss = reconstruction_criterion(X_recon, X_batch)
            class_loss = classification_criterion(y_pred, y_batch)
            
            # Общая потеря (с большим весом для классификации)
            loss = 0.3 * recon_loss + 0.7 * class_loss
            
            # Обратное распространение
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            recon_loss_sum += recon_loss.item()
            class_loss_sum += class_loss.item()
        
        # Вывод статистики каждые 10 эпох
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.4f}, '
                  f'Recon: {recon_loss_sum/len(dataloader):.4f}, '
                  f'Class: {class_loss_sum/len(dataloader):.4f}')
    
    return model

# Пример использования
def main():
    # Предположим, что у нас есть данные в формате DataFrame
    # где каждая строка - признаки для ARP-пакета или окна пакетов
     df = pd.read_csv(r'D:\Проекты\Дипломаня работа\DoFitN\Code\DoFitN\Data\Data_20\combined_features.csv')
    
    # Преобразование данных
     X = df.drop('label', axis=1).values
     y = df['label'].values
    
    # Масштабирование признаков
     scaler = StandardScaler()
     X_scaled = scaler.fit_transform(X)
    
    # Разделение на обучающую и тестовую выборки
     X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    # Обучение модели
     model = train_model(X_train, y_train, input_dim=X_train.shape[1])
    
    # Оценка на тестовой выборке
     X_test_tensor = torch.FloatTensor(X_test)
     _, y_pred = model(X_test_tensor)
     y_pred_np = y_pred.detach().numpy().flatten()
     y_pred_binary = (y_pred_np > 0.5).astype(int)
    
    # Метрики качества
     accuracy = (y_pred_binary == y_test).mean()
     print(f'Test Accuracy: {accuracy:.4f}')
    
     print("Модель создана и готова к обучению.")

if __name__ == "__main__":
    main()

ValueError: could not convert string to float: '4c:5f:70:98:4d:bf'